In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# check data

In [ ]:
train= pd.read_csv("/kaggle/input/taxi-fare-guru-total-amount-prediction-challenge/train.csv")
test= pd.read_csv("/kaggle/input/taxi-fare-guru-total-amount-prediction-challenge/test.csv")

In [ ]:
train.describe()

In [ ]:
test.describe()

In [ ]:
train.info()
test.info()

In [ ]:
test.isna().sum()

summary: train sample --> 175000     test sample --> 50000 

         obj type--> tpep_pickup_datetime, tpep_dropoff_datetime, store_and_fwd_flag, payment_type
         null values in--> passenger_count, RatecodeID, store_and_fwd_flag, congestion_surcharge, Airport_fee 
         null value --> train:6077   test: 1779

# Visualize

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
att_hist= ['VendorID', 'passenger_count', 'RatecodeID', 'store_and_fwd_flag',
       'payment_type','improvement_surcharge', 'total_amount',
       'congestion_surcharge', 'Airport_fee']

n=9
fig,axes= plt.subplots(3, 3, figsize=(15, 15))
axes = axes.ravel()
for i in range(n):
    sns.histplot(train[att_hist[i]], ax=axes[i], bins=20, kde=True)
    axes[i].set_title(f'Column {i+1}')

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

In [ ]:
sns.set(style="whitegrid") 
att_box=['trip_distance','PULocationID','DOLocationID','extra', 'tip_amount','tolls_amount']
num_cols = 6

fig, axes = plt.subplots(2, 3, figsize=(12, 8))
axes = axes.ravel()

for i in range(num_cols):
    sns.boxplot(data=train[att_box[i]], ax=axes[i], width=0.5)
    axes[i].set_title(f'Column {i + 1}')

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

In [ ]:
# when picked and dropped in different days
train['tpep_pickup_datetime']= pd.to_datetime(train['tpep_pickup_datetime'])
train['tpep_dropoff_datetime']=pd.to_datetime(train['tpep_dropoff_datetime'])
train[train['tpep_pickup_datetime'].dt.weekday!= train['tpep_dropoff_datetime'].dt.weekday].shape

In [ ]:
#day[0] and night[1]
pd.to_datetime(train['tpep_pickup_datetime']).apply(lambda x: 0 if 6 <= x.hour < 18 else 1).value_counts()

### Unique and value distriution

train_data['VendorID'].unique() == [1, 0, 2] =={127340,47597,63} 

len(train_data['tpep_pickup_datetime'].unique()) ==109877 

len(train_data['tpep_dropoff_datetime'].unique())==109773 

train_data['passenger_count'].unique() == [ 1., 3., 0., 2., nan, 4., 6., 5., 8., 9.] 

len(train_data['trip_distance'].unique()) ==2929 

train_data['RatecodeID'].unique() ==[1., 99., 3., nan, 2., 5., 4.] 

train_data['store_and_fwd_flag'].unique() == ['N', nan, 'Y'] 

len(train_data['PULocationID'].unique()) == 264 

len(train_data['DOLocationID'].unique()) == 264 

train_data['payment_type'].unique()== ['Credit Card', 'Cash', 'Wallet', 'UPI', 'unknown'] == {135257, 30141, 6077, 2333, 1192} 

len(train_data['extra'].unique()) == 28 

len(train_data['tip_amount'].unique()) == 175000 

len(train_data['tolls_amount'].unique()) == 214  

train_data['improvement_surcharge'].unique() ==[ 1. , 0.3, -1. , 0. ] 

train_data['congestion_surcharge'].unique() == [ 2.5, 0. , nan, -2.5] 

train_data['Airport_fee'].unique() == [0., 175, nan ,-1.75] 

len(train_data['total_amount'].unique())== 8164 

In [ ]:
# Visualize correlation matrix
num_cols = train.select_dtypes(include=['number']).columns
corr_mat = train[num_cols].corr()

plt.figure(figsize=(12, 8))
sns.heatmap(corr_mat, annot=True, cmap='RdYlBu', fmt=".2f")
plt.title('Correlation Matrix for Numerical Features in train set')
plt.show()

# check for outliers
data points out of 1.5 times of iqr are outlier. but outlier must be around 1 in 1000. 

In [ ]:
train['tpep_pickup_datetime']= pd.to_datetime(train['tpep_pickup_datetime'])
train['tpep_dropoff_datetime']=pd.to_datetime(train['tpep_dropoff_datetime'])
train['time_dur']= (train['tpep_dropoff_datetime']- train['tpep_pickup_datetime']).dt.total_seconds().astype(int)
train.shape

In [ ]:
train.describe()

### trip distance

In [ ]:
c=3.61-1.08
iqr = c*1.5
iqr_l= 1.08-iqr
iqr_h= 3.61+iqr
iqr_l,iqr_h

In [ ]:
print(train[train['trip_distance']>30].shape)
test[test['trip_distance']>30].shape

In [ ]:
print(266/175000)
81/50000

### tip amount

In [ ]:
c=7.5-3.47
iqr = c*1.5
iqr_l= 3.47-iqr
iqr_h= 7.5+iqr
iqr_l,iqr_h

In [ ]:
print(train[train['tip_amount']>30].shape)
test[test['tip_amount']>30].shape

In [ ]:
print(305/175000)
90/50000

### tolls amount

In [ ]:
print(train[train['tolls_amount']<0].shape)
test[test['tolls_amount']<0].shape

In [ ]:
print(126/175000)
print(42/50000)

### time duration

In [ ]:
c=3190-(-1171)
iqr= c*1.5
iqr_l= (-1171)-iqr
iqr_h= 3190+iqr
iqr_l,iqr_h

In [ ]:
print(train[train['time_dur']>9731.5].shape)
#test[test['time_dur']>9731.5].shape

In [ ]:
349/175000

decision: distribution of data points value regarding different attributes is same in both train and test. \
          so decided to not to kick any datapoint out

# arrange and drop

In [ ]:
train= pd.read_csv('/kaggle/input/taxi-fare-guru-total-amount-prediction-challenge/train.csv')
train.shape

In [ ]:
from sklearn.preprocessing import LabelEncoder
labelenc= LabelEncoder()

train['store_and_fwd_flag']= labelenc.fit_transform(train['store_and_fwd_flag'])
train['store_and_fwd_flag'].unique()

#date-time dealing
train['tpep_pickup_datetime']= pd.to_datetime(train['tpep_pickup_datetime'])
train['tpep_dropoff_datetime']=pd.to_datetime(train['tpep_dropoff_datetime'])
train['time_dur']= (train['tpep_dropoff_datetime']- train['tpep_pickup_datetime']).dt.total_seconds().astype(int)
train['pick_day']= train['tpep_pickup_datetime'].dt.weekday
train['drop_day']= train['tpep_dropoff_datetime'].dt.weekday
train['day_knight']= train['tpep_pickup_datetime'].apply(lambda x: 0 if 6 <= x.hour < 18 else 1)

train= pd.get_dummies(train, columns=['payment_type'],dtype=int)
print(train.shape)
train.head()

In [ ]:
y= train['total_amount'].copy()
X= train.copy()
X.drop(['tpep_pickup_datetime','tpep_dropoff_datetime','total_amount'],axis=1,inplace=True)
X.head()

In [ ]:
X.isna().sum()

In [ ]:
num_att=['passenger_count','RatecodeID','congestion_surcharge','Airport_fee']

In [ ]:
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [ ]:
#trying to create a scaling
'''from sklearn.preprocessing import StandardScaler

X2= X.copy()
num_att= ['passenger_count','RatecodeID']
imp_scle= ['congestion_surcharge','Airport_fee']
scle= ['extra', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'time_dur']

num_att_pipe=Pipeline([('impute_num',KNNImputer())])
imp_scl_pipe=Pipeline([('imp', KNNImputer()),
                      ('scl', StandardScaler())])
scl_pipe= Pipeline([('sts',StandardScaler())])

ct= ColumnTransformer([('num_miss', num_att_pipe, num_att),
                      ('im_scl', imp_scl_pipe, imp_scle),
                      ('scale', scl_pipe, scle)],
                      remainder="passthrough")

X_tr=ct.fit_transform(X2)
X_tr.shape'''

In [ ]:
num_att_pipe=Pipeline([('impute_num',KNNImputer())])
ct= ColumnTransformer([('num_miss', num_att_pipe, num_att)],
                      remainder="passthrough")

X_tr=ct.fit_transform(X)
X_tr.shape

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test= train_test_split(X_tr,y,test_size=0.2,shuffle=True)
X_train.shape,X_test.shape,y_train.shape,y_test.shape

# Model selection and fit

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score

### Linear regression

In [ ]:
from sklearn.linear_model import LinearRegression
lin_reg= LinearRegression()

In [ ]:
params={'fit_intercept':[True,False]}

grid_cv=GridSearchCV(estimator=lin_reg,
                    param_grid=params,
                    scoring='r2',
                    n_jobs= -1,
                    cv=5,
                    return_train_score=True)

grid_cv.fit(X_train,y_train)

In [ ]:
grid_cv.best_params_

In [ ]:
lin= LinearRegression(fit_intercept= True)
lin.fit(X_train,y_train)

In [ ]:
#train set score
y_train_pred_lin= lin.predict(X_train)
print('train score: ',r2_score(y_train,y_train_pred_lin)) 

#test set score
y_pred_lin= lin.predict(X_test)
print('test score: ',r2_score(y_test,y_pred_lin))

### KNN regressor

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
knn_reg= KNeighborsRegressor()

In [ ]:
'''params={'n_neighbors':[3,5,7],
        'leaf_size':[15,30,45],
        #'algorithm':['auto', 'ball_tree', 'kd_tree', 'brute']
       }

grid_cv=GridSearchCV(estimator=knn_reg,
                    param_grid=params,
                    scoring='r2',
                    n_jobs= -1,
                    cv=3,
                    return_train_score=True)

grid_cv.fit(X_train,y_train)
grid_cv.best_params_ '''

best params: {leaf_size=15, n_neighbors=5}

In [ ]:
knn= KNeighborsRegressor(leaf_size=15, n_neighbors=5)
knn.fit(X_train,y_train)

In [ ]:
#train set score
y_train_pred_knn= knn.predict(X_train)
print('train score: ',r2_score(y_train,y_train_pred_knn)) 

#test set score
y_pred_knn= knn.predict(X_test)
print('test score: ',r2_score(y_test,y_pred_knn))

### random forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

rf_reg= RandomForestRegressor()

In [ ]:
'''params={
       'n_estimators':[100,150,200],
       'min_samples_split':[5,7,10],
       #'min_samples_leaf':[2,3,5,7,10],
       'max_depth':[None, 10, 20],
       #'warm_start':[True,False]
       }

grid_cv=GridSearchCV(estimator=rf_reg,
                    param_grid=params,
                    scoring='r2',
                    n_jobs= -1,
                    cv=3,
                    return_train_score=True)

grid_cv.fit(X_train,y_train)
grid_cv.best_params_'''

{'max_depth': None, 'min_samples_split': 5, 'n_estimators': 200}

In [ ]:
#rf = RandomForestRegressor()
rf= RandomForestRegressor(max_depth=None,n_estimators=200, min_samples_split=5, random_state=42)
rf.fit(X_train, y_train)

In [ ]:
#train set score
y_train_pred_rf= rf.predict(X_train)
print('train score: ',r2_score(y_train,y_train_pred_rf)) 

#test set score
y_pred_rf= rf.predict(X_test)
print('test score: ',r2_score(y_test,y_pred_rf))

### gradient boost

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
gbr_reg= GradientBoostingRegressor()

In [ ]:
'''params={'learning_rate':[0.1,0.3,0.5],
       'n_estimators':[100,150,200],
       'min_samples_split':[5,7,10],
       #'min_samples_leaf':[2,3,5,7,10],
       #'max_depth':[3,5,7],
       #'alpha':[0.5,0.9],
       #'warm_start':[True,False]
       }
gbr_reg= GradientBoostingRegressor()

grid_cv=GridSearchCV(estimator=gbr_reg,
                    param_grid=params,
                    scoring='r2',
                    n_jobs= -1,
                    cv=3,
                    return_train_score=True)

grid_cv.fit(X_train,y_train)
grid_cv.best_params_'''

{'learning_rate': 0.3, 'min_samples_split': 7, 'n_estimators': 150}

In [ ]:
#gbr=GradientBoostingRegressor()
gbr=GradientBoostingRegressor(learning_rate=0.3,n_estimators=150, min_samples_split=7,random_state=42)
gbr.fit(X_train,y_train)

In [ ]:
#train set score
y_train_pred_gbr= gbr.predict(X_train)
print('train score: ',r2_score(y_train,y_train_pred_gbr)) 

#test set score
y_pred_gbr= gbr.predict(X_test)
print('test score: ',r2_score(y_test,y_pred_gbr))

### summary of model fit 
**without scale**       


| model | train score | test score |
| --- | --- | --- |
|Linear Regression | 0.73 | 0.72 |
|KNN Regression| ~0.5| ~0.21|
|KNN + HPT| ~0.5| ~0.21|
|Random forest| 0.99| 0.93|
|Random forest + HPT| 0.988| 0.93~0.94|
|Gradientboost| 0.94| 0.92|
|Gradientboost + HPT| 0.96 |0.92~0.93

**with scale**

| model | train score | test score |
| --- | --- | --- |
|Linear Regression | 0.73 | 0.71 |
|KNN + HPT| 0.86| 0.79|
|Random forest + HPT| 0.987| 0.946|
|Gradientboost + HPT| 0.957 |0.946|

## sample visualize of train-test prediction

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Generate sample data
x_subsampled = np.linspace(0, 10, 10)
y1 = y_train_pred_lin
y2 = y_train_pred_knn
y3 = y_train_pred_rf
y4 = y_train_pred_gbr
y5 = y_train

y1_subsampled = y1[0:10]
y2_subsampled = y2[0:10]
y3_subsampled = y3[0:10]
y4_subsampled = y4[0:10]
y5_subsampled = y5.values[0:10]
plt.figure(figsize=(12, 5))
# Plot the lines
plt.plot(x_subsampled, y1_subsampled, label='Linear regression')
plt.plot(x_subsampled, y2_subsampled, label='KNN regression')
plt.plot(x_subsampled, y3_subsampled, label='Random forest regressor')
plt.plot(x_subsampled, y4_subsampled, label='Gradint boost regressor')
plt.plot(x_subsampled, y5_subsampled, label='y_train')

# Add labels and legend
plt.xlabel('data points')
plt.ylabel('predictions')
plt.title('prediction on train data')
plt.legend()

# Show the plot
plt.show()


In [ ]:
# Generate sample data
x_subsampled = np.linspace(0, 10, 10)
y1 = y_pred_lin
y2 = y_pred_knn
y3 = y_pred_rf
y4 = y_pred_gbr
y5 = y_test

y1_subsampled = y1[0:10]
y2_subsampled = y2[0:10]
y3_subsampled = y3[0:10]
y4_subsampled = y4[0:10]
y5_subsampled = y5.values[0:10]
plt.figure(figsize=(12, 5))
# Plot the lines
plt.plot(x_subsampled, y1_subsampled, label='Linear regression')
plt.plot(x_subsampled, y2_subsampled, label='KNN regression')
plt.plot(x_subsampled, y3_subsampled, label='Random forest regressor')
plt.plot(x_subsampled, y4_subsampled, label='Gradint boost regressor')
plt.plot(x_subsampled, y5_subsampled, label='y_train')

# Add labels and legend
plt.xlabel('data points')
plt.ylabel('predictions')
plt.title('prediction on test data')
plt.legend()

# Show the plot
plt.show()

## choosed model
After considering train_score, test_score and submission_score **Randomforest Regressor** looks most promising. Gradient boosting is second best choice. This maybe because in Randomforest each tree is independent and its handle irrelevant or redundant features better. 

Scaling the data seems to good for models, specially for Linear Regression and KNN Regression. But during final submission in both Randomforest and Gradientboost, submission score decreases. So I decided to go with without scaling.

# submission

In [ ]:
test= pd.read_csv('/kaggle/input/taxi-fare-guru-total-amount-prediction-challenge/test.csv')
test.shape

In [ ]:
from sklearn.preprocessing import LabelEncoder
labelenc= LabelEncoder()

test['store_and_fwd_flag']= labelenc.fit_transform(test['store_and_fwd_flag'])
test['store_and_fwd_flag'].unique()

#date-time dealing
test['tpep_pickup_datetime']= pd.to_datetime(test['tpep_pickup_datetime'])
test['tpep_dropoff_datetime']=pd.to_datetime(test['tpep_dropoff_datetime'])
test['time_dur']= (test['tpep_dropoff_datetime']- test['tpep_pickup_datetime']).dt.total_seconds().astype(int)
test['pick_day']= test['tpep_pickup_datetime'].dt.weekday
test['drop_day']= test['tpep_dropoff_datetime'].dt.weekday
test['day_knight']= test['tpep_pickup_datetime'].apply(lambda x: 0 if 6 <= x.hour < 18 else 1)

test= pd.get_dummies(test, columns=['payment_type'],dtype=int)
test.drop(['tpep_pickup_datetime','tpep_dropoff_datetime'],axis=1,inplace=True)
print(test.shape)
test.head()

In [ ]:
num_att=['passenger_count','RatecodeID','congestion_surcharge','Airport_fee']

In [ ]:
#trying to create a scaling
'''from sklearn.preprocessing import StandardScaler


num_att= ['passenger_count','RatecodeID']
imp_scle= ['congestion_surcharge','Airport_fee']
scle= ['extra', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'time_dur']

num_att_pipe=Pipeline([('impute_num',KNNImputer())])
imp_scl_pipe=Pipeline([('imp', KNNImputer()),
                      ('scl', StandardScaler())])
scl_pipe= Pipeline([('sts',StandardScaler())])

ct= ColumnTransformer([('num_miss', num_att_pipe, num_att),
                      ('im_scl', imp_scl_pipe, imp_scle),
                      ('scale', scl_pipe, scle)],
                      remainder="passthrough")

test_tr=ct.fit_transform(test)
test_tr.shape'''

In [ ]:
num_att_pipe=Pipeline([('impute_num',KNNImputer())])
ct= ColumnTransformer([('num_miss', num_att_pipe, num_att)],
                      remainder="passthrough")

test_tr=ct.fit_transform(test)
test_tr.shape

In [ ]:
predictions= rf.predict(test_tr)
len(predictions)

### scores
**with out scaling**

| model |submission score |
|----|----|
random forest| 0.94038~ 0.94337|
gradiend boosting| 0.93476|

**with scaling**

| model |submission score |
|----|----|
random forest| 0.93476|
gradiend boosting| 0.93065|

In [ ]:
data = {'ID':range(1,50001),
        'total_amount':predictions}

submission= pd.DataFrame(data)
submission.head()

In [ ]:
submission.to_csv("submission.csv",index=None)

#-----------------------------end--------------------------#